In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

In [2]:
super_srag = spark.read.parquet('gs://ai-covid19-datalake/standard/super-srag/super_srag_v1.parquet')

In [3]:
# creating a year attribute
super_srag = super_srag.withColumn('YEAR_SIN_PRI', F.year('DT_SIN_PRI'))

In [4]:
super_srag.limit(2).toPandas()

,NU_NOTIFIC,CS_SEXO,DT_NASC,AGE_AT_NOTIF,AGE_GROUP,CS_GESTANT,CS_RACA,CS_ETINIA,CS_ESCOL_N,SG_UF,CO_MUN_RES,SEM_PRI,SEM_NOT,DIST_PRI_NOTIFIC,DT_SIN_PRI,SG_UF_NOT,CO_MUN_NOT,SURTO_SG,NOSOCOMIAL,AVE_SUINO,VACINA,HOSPITAL,DIST_PRI_INTERNA,SUPORT_VEN,UTI,DIST_PRI_ENTUTI,CLASSI_OUT,CRITERIO,EVOLUCAO,CLASSI_FIN,SYMP_GROUP1,SYMP_GROUP2,SYMP_GROUP3,SYMP_GROUP4,OUTRO_SIN,OUTRO_DES,RF_GROUP1,RF_GROUP2,RF_GROUP3,RF_GROUP4,OBES_IMC,OUT_MORBI,MORB_DESC,RAIOX_RES,DIST_PRI_RAIOX,TOMO_RES,DIST_PRI_TOMO,AMOSTRA,TP_AMOSTRA,DT_COLETA,DIST_PRI_COLETA,PP_IF_RESUL,PP_TRA_RESUL,DIST_PRI_TRA,PP_PCR_RESUL,DIST_PRI_PCR,PP_RES_SOR_IGA,PP_RES_SOR_IGM,PP_RES_SOR_IGG,DIST_PRI_SOR,DIST_PRI_IF,DIST_PRI_NOTIFIC_Q,DIST_PRI_INTERNA_Q,DIST_PRI_ENTUTI_Q,DIST_PRI_SAIDUTI_Q,DIST_PRI_EVOLUCA_Q,DIST_PRI_ENCERRA_Q,DIST_PRI_RAIOX_Q,DIST_PRI_TOMO_Q,DIST_PRI_COLETA_Q,DIST_PRI_SOR_Q,DIST_PRI_PCR_Q,DIST_PRI_TRA_Q,DIST_PRI_IF_Q,EPI_WEEK_YEAR,GMR_TRANSIT_STATIONS_AVG,GMR_GROCERY_AND_PHARMACY_AVG,GMR_RETAIL_AND_RECREATION_AVG,GMR_WORKPLACES_PERCENT_AVG,GMR_RESIDENTIAL_PERCENT_AVG,GMR_PARKS_PERCENT_AVG,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_AVG,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_AVG,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_AVG,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_AVG,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_AVG,GMR_PARKS_PERCENT_1WEEK_BEFORE_AVG,GMR_TRANSIT_STATIONS_2WEEKS_AVG,GMR_GROCERY_AND_PHARMACY_2WEEKS_AVG,GMR_RETAIL_AND_RECREATION_2WEEKS_AVG,GMR_WORKPLACES_PERCENT_2WEEKS_AVG,GMR_RESIDENTIAL_PERCENT_2WEEKS_AVG,GMR_PARKS_PERCENT_2WEEKS_AVG,INMET_TEMP_C_AVG,INMET_RELATIVE_AIR_HUMIDITY_AVG,INMET_DAILY_PRECIPT_AVG,INMET_TEMP_C_1WEEK_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_AVG,INMET_DAILY_PRECIPT_1WEEK_BEFORE_AVG,INMET_TEMP_C_2WEEKS_BEFORE_AVG,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_AVG,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_AVG,GMR_TRANSIT_STATIONS_Q,GMR_GROCERY_AND_PHARMACY_Q,GMR_RETAIL_AND_RECREATION_Q,GMR_WORKPLACES_PERCENT_Q,GMR_RESIDENTIAL_PERCENT_Q,GMR_PARKS_PERCENT_Q,GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q,GMR_GROCERY_AND_PHARMACY_1WEEK_BEFORE_Q,GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q,GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q,GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q,GMR_PARKS_PERCENT_1WEEK_BEFORE_Q,GMR_TRANSIT_STATIONS_2WEEKS_Q,GMR_GROCERY_AND_PHARMACY_2WEEKS_Q,GMR_RETAIL_AND_RECREATION_2WEEKS_Q,GMR_WORKPLACES_PERCENT_2WEEKS_Q,GMR_RESIDENTIAL_PERCENT_2WEEKS_Q,GMR_PARKS_PERCENT_2WEEKS_Q,INMET_TEMP_C_Q,INMET_RELATIVE_AIR_HUMIDITY_Q,INMET_DAILY_PRECIPT_Q,INMET_TEMP_C_1WEEK_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q,INMET_DAILY_PRECIPT_1WEEK_BEFORE_Q,INMET_TEMP_C_2WEEKS_BEFORE_Q,INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q,INMET_DAILY_PRECIPT_2WEEKS_BEFORE_Q,YEAR_SIN_PRI
0,68719768275,M,1967-01-25,54,7,6,4,None,NaN,BA,290240,19,20,7,2021-05-15,BA,291800,None,NaN,NaN,None,1,NaN,NaN,NaN,None,None,None,None,None,9,3,2,1,NaN,None,1,1,1,1,None,None,None,None,None,None,None,2,NaN,None,NaN,9,9,None,6,None,4,4,4,None,None,3,6,6,6,6,6,6,6,6,6,6,6,6,19-2021,-26.208333,31.788462,-24.079137,-0.099905,7.358333,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021
1,68719899483,M,1959-10-20,61,8,6,4,None,9.0,BA,292050,19,20,8,2021-05-11,BA,292050,None,2.0,2.0,None,1,8.0,3.0,2.0,None,None,None,None,None,3,1,3,1,1.0,ASTENIA E INAPETENCIA,1,1,1,1,None,None,None,None,None,6,None,1,1.0,2021-05-17,6.0,9,9,None,7,None,4,4,4,None,None,3,4,6,6,6,6,6,6,4,6,6,6,6,19-2021,-26.208333,31.788462,-24.079137,-0.099905,7.358333,-44.62533,-25.097436,34.628571,-20.930314,0.773764,6.81768,-44.472296,-31.233161,17.257009,-30.477193,-4.605239,8.130556,-46.526854,22.265185,71.299468,1.115789,23.258355,74.984254,2.16,24.092008,74.733094,1.653012,3,5,3,4,2,2,4,5,4,5,2,2,3,4,3,3,3,1,2,3,2,3,4,3,4,4,2,2021


In [5]:
# criar PP_RES_SOR

In [6]:
cols = [
        'NU_NOTIFIC', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO', 
        'AGE_AT_NOTIF', 'AGE_GROUP', 'DIST_PRI_NOTIFIC_Q', 'EPI_WEEK_YEAR', 'YEAR_SIN_PRI', 'SG_UF',
        'DIST_PRI_INTERNA_Q', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_SAIDUTI_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q',
        'SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
        'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3', 'RF_GROUP4', 
        'SUPORT_VEN', 'UTI', 'HOSPITAL',
        'DIST_PRI_RAIOX_Q', 'DIST_PRI_TOMO_Q', 'DIST_PRI_IF_Q', 'DIST_PRI_TRA_Q', 'DIST_PRI_PCR_Q', 'DIST_PRI_SOR_Q',
        'RAIOX_RES', 'TOMO_RES', 'PP_IF_RESUL', 'PP_TRA_RESUL', 'PP_PCR_RESUL', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGG',
        'GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q', 'GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q', 'GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q', 'GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q',
        'GMR_TRANSIT_STATIONS_2WEEKS_Q', 'GMR_RETAIL_AND_RECREATION_2WEEKS_Q', 'GMR_RESIDENTIAL_PERCENT_2WEEKS_Q', 'GMR_WORKPLACES_PERCENT_2WEEKS_Q',
        'INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q', 'INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q'
       ]

In [7]:
# showing some piece of descriptive analysis for sorological
super_srag.select(['PP_RES_SOR_IGA', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGG']).groupBy(['PP_RES_SOR_IGA', 'PP_RES_SOR_IGA', 'PP_RES_SOR_IGG']).count().show()

+--------------+--------------+--------------+-------+
|PP_RES_SOR_IGA|PP_RES_SOR_IGA|PP_RES_SOR_IGG|  count|
+--------------+--------------+--------------+-------+
|             9|             9|             4|     34|
|             9|             9|             5|      5|
|             4|             4|             1|  88203|
|             4|             4|             2|  63411|
|             5|             5|             4|     31|
|             1|             1|             4|   1763|
|             5|             5|             1|     80|
|             5|             5|             5|     52|
|             1|             1|             1|    802|
|             2|             2|             3|      4|
|             9|             9|             2|    840|
|             4|             4|             4|1946930|
|             3|             3|             3|      7|
|             2|             2|             2|   1201|
|             9|             9|             3|      2|
|         

In [8]:
super_srag.select('PP_PCR_RESUL', 'YEAR_SIN_PRI', 'CLASSI_FIN').filter(F.col('PP_PCR_RESUL') == '2').groupBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).count().orderBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).show()

+------------+----------+------+
|YEAR_SIN_PRI|CLASSI_FIN| count|
+------------+----------+------+
|        2019|         5|     1|
|        2020|         5|487262|
|        2021|         5|365946|
+------------+----------+------+



In [9]:
super_srag.select('PP_IF_RESUL', 'YEAR_SIN_PRI', 'CLASSI_FIN').filter(F.col('PP_IF_RESUL') == '2').groupBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).count().orderBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).show()

+------------+----------+-----+
|YEAR_SIN_PRI|CLASSI_FIN|count|
+------------+----------+-----+
|        2020|      null|   21|
|        2020|         2|    2|
|        2020|         5| 5779|
|        2021|      null|  300|
|        2021|         1|    7|
|        2021|         2|    9|
|        2021|         3|    1|
|        2021|         5| 7814|
+------------+----------+-----+



In [10]:
super_srag.select('PP_TRA_RESUL', 'YEAR_SIN_PRI', 'CLASSI_FIN').filter(F.col('PP_TRA_RESUL') == '2').groupBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).count().orderBy(['YEAR_SIN_PRI', 'CLASSI_FIN']).show()

+------------+----------+-----+
|YEAR_SIN_PRI|CLASSI_FIN|count|
+------------+----------+-----+
|        2020|      null|  126|
|        2020|         1|    4|
|        2020|         2|   10|
|        2020|         3|    4|
|        2020|         5|16885|
|        2021|      null| 2465|
|        2021|         1|   64|
|        2021|         2|   22|
|        2021|         3|    2|
|        2021|         5|77543|
+------------+----------+-----+



In [11]:
# not covid dataset 
super_srag.select(['CLASSI_FIN', 'YEAR_SIN_PRI']).filter((F.col('CLASSI_FIN') ==  '1') | (F.col('CLASSI_FIN') ==  '2')).groupBy('YEAR_SIN_PRI').count().orderBy('YEAR_SIN_PRI').show()

+------------+-----+
|YEAR_SIN_PRI|count|
+------------+-----+
|        2018|   21|
|        2019|16278|
|        2020| 7017|
|        2021| 5107|
+------------+-----+



<hr />
<hr />
<hr />

# Dataset 1: 
## Selecting for covid-19 confirmed cases only those with positive RT/PCR results
### data balancing with stratified sampling strategy using the col: EPI_WEEK_YEAR (with 0.05 of fraction for each age group)

In [12]:
count = super_srag.filter(F.col('PP_PCR_RESUL') == '2').count()
super_srag.filter(F.col('PP_PCR_RESUL') == '2')\
          .select(['EPI_WEEK_YEAR']).groupBy('EPI_WEEK_YEAR').count().orderBy('count')\
          .withColumn('%', (F.col('count')/count)*100).show()

+-------------+-----+--------------------+
|EPI_WEEK_YEAR|count|                   %|
+-------------+-----+--------------------+
|      13-2019|    1|1.172045770731438...|
|      08-2020|   26|0.003047319003901...|
|      09-2020|   42|0.004922592237072...|
|      10-2020|  213|0.024964574916579642|
|      20-2021|  505| 0.05918831142193765|
|      11-2020| 1009| 0.11825941826680214|
|      12-2020| 3249| 0.38079767091064437|
|      13-2020| 4887|  0.5727787681564541|
|      19-2021| 5417|  0.6348971940052203|
|      42-2020| 6118|  0.7170576025334942|
|      14-2020| 6128|  0.7182296483042255|
|      43-2020| 6289|  0.7370995852130018|
|      44-2020| 6438|  0.7545630671969001|
|      41-2020| 6553|  0.7680415935603117|
|      40-2020| 6925|  0.8116416962315213|
|      39-2020| 7243|  0.8489127517407811|
|      38-2020| 7559|  0.8859493980958943|
|      37-2020| 8341|  0.9776033773670929|
|      15-2020| 8485|  0.9944808364656256|
|      45-2020| 8839|  1.0359712567495187|
+----------

In [13]:
epi_weeks = super_srag.select('EPI_WEEK_YEAR').distinct().rdd.flatMap(lambda x: x).collect()

fractions = {}

for e_w in epi_weeks:
    fractions[e_w] = 0.05

df_3_covid = super_srag.filter(F.col('PP_PCR_RESUL') == '2').sampleBy('EPI_WEEK_YEAR', fractions=fractions, seed=2021)

In [14]:
count = df_3_covid.filter(F.col('PP_PCR_RESUL') == '2').count()
print(count)
df_3_covid.filter(F.col('PP_PCR_RESUL') == '2')\
          .select(['EPI_WEEK_YEAR']).groupBy('EPI_WEEK_YEAR').count().orderBy('count')\
          .withColumn('%', (F.col('count')/count)*100).show()

42484
+-------------+-----+--------------------+
|EPI_WEEK_YEAR|count|                   %|
+-------------+-----+--------------------+
|      08-2020|    1|0.002353827323227568|
|      09-2020|    2|0.004707654646455136|
|      10-2020|    6|0.014122963939365409|
|      20-2021|   26| 0.06119951040391676|
|      11-2020|   50|  0.1176913661613784|
|      12-2020|  164| 0.38602768100932117|
|      13-2020|  233|  0.5484417663120233|
|      19-2021|  291|  0.6849637510592222|
|      43-2020|  294|   0.692025233028905|
|      42-2020|  308|  0.7249788155540909|
|      39-2020|  319|  0.7508709161095942|
|      44-2020|  322|  0.7579323980792769|
|      14-2020|  324|  0.7626400527257321|
|      41-2020|  327|  0.7697015346954148|
|      38-2020|  358|  0.8426701817154693|
|      40-2020|  369|  0.8685622822709725|
|      37-2020|  388|  0.9132850014122964|
|      15-2020|  423|  0.9956689577252613|
|      36-2020|  451|  1.0615761227756333|
|      45-2020|  467|  1.0992373599472742|
+----

In [15]:
count = df_3_covid.filter(F.col('PP_PCR_RESUL') == '2').count()
print(count)
df_3_covid.filter(F.col('PP_PCR_RESUL') == '2')\
          .select(['AGE_GROUP']).groupBy('AGE_GROUP').count().orderBy('AGE_GROUP')\
          .withColumn('%', (F.col('count')/count)*100).show()

# 42478
# +---------+-----+-------------------+
# |AGE_GROUP|count|                  %|
# +---------+-----+-------------------+
# |        1|  232| 0.5461650736852017|
# |        2|  201|0.47318611987381703|
# |        3|  307| 0.7227270587127455|
# |        4| 1464|  3.446489947737653|
# |        5| 4105|  9.663825980507557|
# |        6| 6267|  14.75351946890155|
# |        7| 8314| 19.572484580253306|
# |        8| 9049| 21.302792033523236|
# |        9| 7101| 16.716888742407836|
# |       10| 4256| 10.019304110363011|
# |       11| 1117| 2.6295964970102172|
# |       12|   65|0.15302038702387116|
# +---------+-----+-------------------+

42484
+---------+-----+-------------------+
|AGE_GROUP|count|                  %|
+---------+-----+-------------------+
|        1|  193| 0.4542886733829206|
|        2|  170| 0.4001506449486865|
|        3|  326| 0.7673477073721872|
|        4| 1430| 3.3659730722154224|
|        5| 4042|   9.51417004048583|
|        6| 6325|  14.88795781941437|
|        7| 8271| 19.468505790415218|
|        8| 9202|  21.65991902834008|
|        9| 7155|  16.84163449769325|
|       10| 4219|  9.930797476697109|
|       11| 1100|  2.589210055550325|
|       12|   51|0.12004519348460596|
+---------+-----+-------------------+



In [16]:
df_3_not_covid = super_srag.filter((F.col('CLASSI_FIN') ==  '1') | (F.col('CLASSI_FIN') ==  '2')).withColumn('CLASSI_FIN', F.lit('1'))

In [17]:
df_3_not_covid.count()

28423

In [18]:
df_3 = df_3_not_covid.union(df_3_covid)

In [19]:
df_3.select('CLASSI_FIN').groupBy('CLASSI_FIN').count().show()

+----------+-----+
|CLASSI_FIN|count|
+----------+-----+
|         5|42484|
|         1|28423|
+----------+-----+



In [20]:
df_3 = df_3.select(cols).persist()

<hr />
<hr />
<hr />

## Preprocessing Dataset 1
### some description before the script

In [21]:
for col in cols: 
    if col != "NU_NOTIFIC":
        
        print("==== " + col + " ====")
        
        df_3.select(col).printSchema()
        
        print("GroupBy: all")
        
        count = df_3.count()
        df_3.select([col]).groupBy(col).count().orderBy(col)\
            .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: covid")
        
        count = df_3.filter(F.col('CLASSI_FIN') == '5').count()
        df_3.filter(F.col('CLASSI_FIN') == '5').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: not covid")
        count = count = df_3.filter(F.col('CLASSI_FIN') == '1').count()
        df_3.filter(F.col('CLASSI_FIN') == '1').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print('=========================')
        print('=========================')

==== CLASSI_FIN ====
root
 |-- CLASSI_FIN: string (nullable = true)

GroupBy: all
+----------+-----+------------------+
|CLASSI_FIN|count|                 %|
+----------+-----+------------------+
|         1|28423|40.156823961571064|
|         5|42357|59.843176038428936|
+----------+-----+------------------+

GroupBy: covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         5|42357|100.0|
+----------+-----+-----+

GroupBy: not covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         1|28423|100.0|
+----------+-----+-----+

==== CRITERIO ====
root
 |-- CRITERIO: string (nullable = true)

GroupBy: all
+----------+-----+--------------------+
|  CRITERIO|count|                   %|
+----------+-----+--------------------+
|      null| 1653|  2.3354054817745125|
|         1|68499|   96.77733823113874|
|         2|  143| 0.20203447301497598|
|24/11/2020|    1|0.001412828482622...|
|         3|  360|  0.5086182537439954|
|    

### Casting variables

In [22]:
cols_to_cast = ['RAIOX_RES', 'UTI', 'SUPORT_VEN', 'YEAR_SIN_PRI', 'AGE_GROUP']

In [23]:
for col in cols_to_cast:
    df_3 = df_3.withColumn(col, F.col(col).cast('string'))

### Filling nulls

In [24]:
# null to "9"
cols_null_to_9 = ['TOMO_RES', 'RAIOX_RES', 'HOSPITAL', 'UTI', 'SUPORT_VEN', 'EVOLUCAO', 'CRITERIO']
for col in cols_null_to_9:
    df_3 = df_3.withColumn(col, F.when(F.col(col).isNull(), '9').otherwise(F.col(col)))

In [25]:
# null to string

In [26]:
cols_null_to_string = ['SG_UF']
for col in cols_null_to_string:
    df_3 = df_3.withColumn(col, F.when(F.col(col).isNull(), 'NULL').otherwise(F.col(col)))

### Taking out invalid values

In [27]:
df_3 = df_3.filter(F.col('TOMO_RES') != '20/11/2020')
df_3 = df_3.filter(F.col('HOSPITAL') != 'RJ')

### Showing final descriptive

In [28]:
for col in cols: 
    if col != "NU_NOTIFIC":
        
        print("==== " + col + " ====")
        
        df_3.select(col).printSchema()
        
        print("GroupBy: all")
        
        count = df_3.count()
        df_3.select([col]).groupBy(col).count().orderBy(col)\
            .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: covid")
        
        count = df_3.filter(F.col('CLASSI_FIN') == '5').count()
        df_3.filter(F.col('CLASSI_FIN') == '5').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print("GroupBy: not covid")
        count = count = df_3.filter(F.col('CLASSI_FIN') == '1').count()
        df_3.filter(F.col('CLASSI_FIN') == '1').select([col]).groupBy(col).count().orderBy(col)\
                                               .withColumn('%', (F.col('count')/count)*100).show()
        
        print('=========================')
        print('=========================')

==== CLASSI_FIN ====
root
 |-- CLASSI_FIN: string (nullable = true)

GroupBy: all
+----------+-----+------------------+
|CLASSI_FIN|count|                 %|
+----------+-----+------------------+
|         1|28422|40.155978468189716|
|         5|42357|59.844021531810284|
+----------+-----+------------------+

GroupBy: covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         5|42357|100.0|
+----------+-----+-----+

GroupBy: not covid
+----------+-----+-----+
|CLASSI_FIN|count|    %|
+----------+-----+-----+
|         1|28422|100.0|
+----------+-----+-----+

==== CRITERIO ====
root
 |-- CRITERIO: string (nullable = true)

GroupBy: all
+--------+-----+-------------------+
|CRITERIO|count|                  %|
+--------+-----+-------------------+
|       1|68499|  96.77870554825584|
|       2|  143|0.20203732745588382|
|       3|  360|  0.508625439749078|
|       4|  124|0.17519320702468244|
|       9| 1653| 2.3354384775145167|
+--------+-----+-------------

<hr />
<hr />
<hr />

In [ ]:
df = {}

df_keys = range(0,500)

for df_key in df_keys:
    
    # establishing the proportion for each category on EPI_WEEK_YEAR attribute
    epi_weeks = super_srag.select('EPI_WEEK_YEAR').distinct().rdd.flatMap(lambda x: x).collect()

    fractions = {}

    for e_w in epi_weeks:
        fractions[e_w] = 0.05
    
    # getting stratified sample for super srag by AGE_GROUP when PCR_RESULT is positive for covid
    df_covid = super_srag.filter(F.col('PP_PCR_RESUL') == '2')\
                         .sampleBy('EPI_WEEK_YEAR', fractions=fractions, seed=df_key)
    
    # getting all NOT COVID records from super srag
    df_not_covid = super_srag.filter((F.col('CLASSI_FIN') ==  '1') | (F.col('CLASSI_FIN') ==  '2'))\
                             .withColumn('CLASSI_FIN', F.lit('1'))
    
    # unioning the datasets and putting into a sample dataset to be written and submitted to the model
    df[df_key] = df_not_covid.union(df_covid)

    
    # writing the sample dataset
    pad_key = str(df_key).zfill(3)
    id_dataset = 'positivepcrforcovid'
    
    df[df_key].write.parquet('gs://ai-covid19-datalake/trusted/XGB-StratifiedSampleDatasets-3/dataset_3_'+id_dataset+'_'+pad_key+'.parquet', mode='overwrite')
#     df[df_key].coalesce(1).write.csv('gs://ai-covid19-datalake/trusted/XGB-StratifiedSampleDatasets-1/dataset_1_'+id_dataset+'_'+pad_key+'.csv', header='true'

In [ ]:
print('finished')

finished


In [31]:
super_srag.unpersist()

DataFrame[NU_NOTIFIC: bigint, CS_SEXO: string, DT_NASC: date, AGE_AT_NOTIF: bigint, AGE_GROUP: int, CS_GESTANT: int, CS_RACA: int, CS_ETINIA: string, CS_ESCOL_N: int, SG_UF: string, CO_MUN_RES: int, SEM_PRI: string, SEM_NOT: string, DIST_PRI_NOTIFIC: int, DT_SIN_PRI: date, SG_UF_NOT: string, CO_MUN_NOT: string, SURTO_SG: int, NOSOCOMIAL: int, AVE_SUINO: int, VACINA: int, HOSPITAL: string, DIST_PRI_INTERNA: int, SUPORT_VEN: int, UTI: int, DIST_PRI_ENTUTI: int, CLASSI_OUT: string, CRITERIO: string, EVOLUCAO: string, CLASSI_FIN: int, SYMP_GROUP1: string, SYMP_GROUP2: string, SYMP_GROUP3: string, SYMP_GROUP4: string, OUTRO_SIN: int, OUTRO_DES: string, RF_GROUP1: string, RF_GROUP2: string, RF_GROUP3: string, RF_GROUP4: string, OBES_IMC: string, OUT_MORBI: int, MORB_DESC: string, RAIOX_RES: int, DIST_PRI_RAIOX: int, TOMO_RES: string, DIST_PRI_TOMO: int, AMOSTRA: int, TP_AMOSTRA: int, DT_COLETA: date, DIST_PRI_COLETA: int, PP_IF_RESUL: string, PP_TRA_RESUL: string, DIST_PRI_TRA: int, PP_PCR_R